In [23]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [24]:
def cleanData(df,substance):
    df = df.drop(columns=['historianTagnummer'])
    df.rename(columns={'hstWaarde': substance + 'Value'}, inplace=True)
    df[substance + 'Value'] = df[substance + 'Value'].astype(float)
    df = df.drop(columns=['datumBeginMeting'])
    df['datumEindeMeting'] = pd.to_datetime(df['datumEindeMeting'])
    df.rename(columns={'datumEindeMeting': 'measurementDate'}, inplace=True)
    df.to_parquet('../../data/cleanedData/'+ substance +'.parquet', index=False)
    pass

In [25]:
def shiftDates(df, name):
    duplicates = df[df.duplicated(subset='measurementDate', keep=False)].index.tolist()
    marchDate = df.loc[df['measurementDate'] == '2021-03-28 03:00:00']
    startTime = marchDate.index.values[0]
    endTime = duplicates[-1]

    for i in range(startTime, endTime + 1):
        if i < (endTime - len(duplicates)):
            df.loc[i, 'measurementDate'] -= timedelta(hours=1)
        elif not ((i % 2) == 0):
            df.loc[i, 'measurementDate'] -= timedelta(hours=1)
    
    df = df.sort_values(by='measurementDate')
    df.to_parquet('../../data/shiftedDates/'+ name +'.parquet', index=False)
    pass

In [ ]:
# from joblib import Parallel, delayed
# from datetime import timedelta

# def shift_date(row, duplicates):
#     if row.name < (endTime - len(duplicates)):
#         row['measurementDate'] -= timedelta(hours=1)
#     elif not ((row.name % 2) == 0):
#         row['measurementDate'] -= timedelta(hours=1)
#     return row

# def shift_dates(df, name):
#     duplicates = df[df.duplicated(subset='measurementDate', keep=False)].index.tolist()
#     march_date = df.loc[df['measurementDate'] == '2021-03-28 03:00:00']
#     start_time = march_date.index.values[0]
#     end_time = duplicates[-1]

#     num_cores = -1  # Use all available cores
#     results = Parallel(n_jobs=num_cores)(
#         delayed(shift_date)(row, duplicates) for _, row in df.iterrows()
#     )

#     df = pd.DataFrame(results)
#     df = df.sort_values(by='measurementDate')
#     df.to_parquet(f'../../data/shiftedDates/{name}.parquet', index=False)
